In [ ]:
from pyspark.sql import SparkSession
from pyspark.sql.types import *
from pyspark.sql import functions as F
from pyspark.sql import Window
from delta.tables import DeltaTable

# Create SparkSession
spark = SparkSession.builder.appName("DeltaSession") \
            .config("spark.jars.packages", "io.delta:delta-core_2.12:2.3.0") \
            .config("spark.sql.extensions", "io.delta.sql.DeltaSparkSessionExtension") \
            .config("spark.sql.catalog.spark_catalog", "org.apache.spark.sql.delta.catalog.DeltaCatalog")\
            .getOrCreate()

In [ ]:
# Define the schema for a DataFrame
schema = StructType([
    StructField("id", IntegerType(), True),
    StructField("name", StringType(), True),
    StructField("age", IntegerType(), True),
    StructField("city", StringType(), True)
])

# Create a DataFrame using the schema
data = [(1, "Alice", 25, "New York"), 
        (2, "Bob", 30, "San Francisco"), 
        (3, "Charlie", 35, "Chicago")]

df = spark.createDataFrame(data, schema)

# Show the DataFrame
df.toPandas()

In [ ]:
df.write.format("delta").saveAsTable("09table_constraints")

In [ ]:
spark.sql("SHOW TBLPROPERTIES 09table_constraints").toPandas()

# Existing Table

## Add Constraint

In [ ]:
spark.sql("""
    ALTER TABLE 09table_constraints
    ADD CONSTRAINT positive_age CHECK (age > 0)
""")

In [ ]:
spark.sql("SHOW TBLPROPERTIES 09table_constraints").toPandas()

## Append New Data

In [ ]:
# Define the schema for a DataFrame
schema = StructType([
    StructField("id", IntegerType(), True),
    StructField("name", StringType(), True),
    StructField("age", IntegerType(), True),
    StructField("city", StringType(), True),
])

# Create a DataFrame using the schema
data = [(4, "Oliveira", 25, "New York")]

df = spark.createDataFrame(data, schema)

# Show the DataFrame
df.toPandas()

In [ ]:
(df.write
    .mode("append")
    .format("delta")
    .saveAsTable("09table_constraints")
)

In [ ]:
df = spark.read.format("delta").table("09table_constraints")

df.toPandas()

## Append New Data with Constraint Blocking

In [ ]:
# Define the schema for a DataFrame
schema = StructType([
    StructField("id", IntegerType(), True),
    StructField("name", StringType(), True),
    StructField("age", IntegerType(), True),
    StructField("city", StringType(), True),
])

# Create a DataFrame using the schema
data = [(5, "Fernando", -5, "New York")]

df = spark.createDataFrame(data, schema)

# Show the DataFrame
df.toPandas()

In [ ]:
(df.write
    .mode("append")
    .format("delta")
    .saveAsTable("09table_constraints")
)

In [ ]:
df = spark.read.format("delta").table("09table_constraints")

df.toPandas()

# New Table

In [ ]:
spark.sql("""
CREATE OR REPLACE TABLE 09table_constraints2 (
   id INT,
   name STRING NOT NULL,
   age INT,
   city STRING
 ) USING DELTA;
""")

In [ ]:
spark.sql("SHOW TBLPROPERTIES 09table_constraints2").toPandas()

In [ ]:
spark.sql("""
INSERT INTO 09table_constraints2 (id, name, age, city)
VALUES (5, NULL, 5, 'Sao Paulo')
""")

In [ ]:
spark.sql("""
SELECT * FROM 09table_constraints2
""").toPandas()